In [2]:
import pandas as pd
import numpy as np

In [12]:
titanic = pd.read_csv('train.csv') # can set index_col=0 if want first column in file to be the indexti
titanic.head()
print(titanic.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [13]:
df=pd.DataFrame(titanic[['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked']])
df.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked
0,1,0,3,male,22.0,7.2500,S
1,2,1,1,female,38.0,71.2833,C
2,3,1,3,female,26.0,7.9250,S
3,4,1,1,female,35.0,53.1000,S
4,5,0,3,male,35.0,8.0500,S


In [5]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
Fare             0
Embarked         2
dtype: int64

In [6]:
df[df['Embarked'].isna()]

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked
61,62,1,1,female,38.0,80.0,NaN
829,830,1,1,female,62.0,80.0,NaN


In [8]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [15]:
df['Embarked'].fillna(value='X',inplace=True)
df.isna().sum()
#df=df[df['Embarked'].notna()]
#df=df.dropna(subset=['Embarked'])
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
Fare             0
Embarked         0
dtype: int64

In [18]:
#df.groupby('Pclass')['Age'].median()
df.groupby(['Pclass']).median()['Age']

Pclass
1    37.0
2    29.0
3    24.0
Name: Age, dtype: float64

In [22]:
df['impAge']=df[['Pclass','Age']].groupby('Pclass').transform(lambda x:x.fillna(x.median()))['Age']# 仔细看看
df.head()
#df.isna().sum()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,impAge
0,1,0,3,male,22.0,7.2500,S,22.0
1,2,1,1,female,38.0,71.2833,C,38.0
2,3,1,3,female,26.0,7.9250,S,26.0
3,4,1,1,female,35.0,53.1000,S,35.0
4,5,0,3,male,35.0,8.0500,S,35.0


In [31]:
df.pivot_table(index='Pclass', columns = 'Sex',values='PassengerId',aggfunc='count',margins=True)

Sex,female,male,All
Pclass,,,
1,94,122,216
2,76,108,184
3,144,347,491
All,314,577,891


In [33]:
df.pivot_table(index='Pclass', columns = 'Sex',values='Survived',aggfunc='mean',margins=True)

Sex,female,male,All
Pclass,,,
1,0.968085,0.368852,0.629630
2,0.921053,0.157407,0.472826
3,0.500000,0.135447,0.242363
All,0.742038,0.188908,0.383838


In [34]:
df['impAge'].describe()

count    891.000000
mean      29.066409
std       13.244532
min        0.420000
25%       22.000000
50%       26.000000
75%       37.000000
max       80.000000
Name: impAge, dtype: float64

In [43]:
def getAgeGroup(x):
    if x<=22:
        return '0-[,22]'
    elif x>22 and x<=26:
        return '1-(22,26]'
    elif x>26 and x<=36:
        return '2-(26,36]'
    elif x>36:
        return '3-(36,]'
df['AgeGroup']=df['impAge'].apply(getAgeGroup)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,impAge,AgeGroup
0,1,0,3,male,22.0,7.2500,S,22.0,"0-[,22]"
1,2,1,1,female,38.0,71.2833,C,38.0,"3-(36,]"
2,3,1,3,female,26.0,7.9250,S,26.0,"1-(22,26]"
3,4,1,1,female,35.0,53.1000,S,35.0,"2-(26,36]"
4,5,0,3,male,35.0,8.0500,S,35.0,"2-(26,36]"


In [44]:
df.pivot_table(index='AgeGroup', columns = 'Sex',values='PassengerId',aggfunc='count',margins=True)

Sex,female,male,All
AgeGroup,,,
"0-[,22]",96,135,231
"1-(22,26]",73,151,224
"2-(26,36]",70,141,211
"3-(36,]",75,150,225
All,314,577,891


In [45]:
df.pivot_table(index='AgeGroup', columns = 'Sex',values='Survived',aggfunc='mean',margins=True)

Sex,female,male,All
AgeGroup,,,
"0-[,22]",0.697917,0.229630,0.424242
"1-(22,26]",0.657534,0.119205,0.294643
"2-(26,36]",0.842857,0.234043,0.436019
"3-(36,]",0.786667,0.180000,0.382222
All,0.742038,0.188908,0.383838
